In [2]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import ticker
import seaborn as sns
from astropy.io import fits
import pandas as pd
from scipy.interpolate import interp1d, RectBivariateSpline
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage import rotate
from frank.utilities import UVDataBinner
from frank.plot import plot_vis_quantity
from scipy.integrate import trapz, simps

def load_profile(disk):
    """read radial profile from frank ; converts [r] from arcsec to radians [intensity] is Jy/sr"""

    if disk.startswith(('/data/modeldata','/data/C4C7', '/data/C5C8', '/data/C6C9', '/data/band')):
        # takes non-negative profile by default
        filename = f'{disk}_frank_profile_fit_nn.txt'
    
    else:
        filename = '/data/es833/dataforfrank/uv_tables/' + disk + '_selfcal_cont_bin60s_1chan_nopnt_nofl_frank_profile_fit.txt'
    
    profile = pd.read_csv(filename, header = None, skiprows = 1, delim_whitespace=True, names = ['r','I','I_unc'])
    r,intensity = profile['r'].to_numpy(), profile['I'].to_numpy()
    r = r*np.pi/(180*60*60) # convert from arsec to rad

    return r, intensity

In [ ]:
def pretty_image(disk, r_planet, SA, a_or_i, Mth, dustnum, hand, nxy=1024, rmax=None, axisym=False, project=False, log=False, png=False, size=13, band=7):
    """
    makes a pretty image of disk+spiral: 
    -weak spirals look best with linear colormap, 
    -strong spirals wash out the image so look better with log colormap

    rmax = max radius to plot to ; nxy = no. pixels
    [r_planet] = [rmax] = arcsec, [SA] = deg
    """

    # profile
    r, intens = load_profile(disk)
    rad2arcsec = 180*60*60/np.pi
    r *= rad2arcsec
    intens /= rad2arcsec**2 # now [I] = Jy / (arcsec)**2

    if project:
        # disk geometry
        inc, PA, _, _ = load_geo(disk)
        inc *= np.pi/180
        # convert inc to rad, leave PA in degrees as scipy.ndimage.rotate wants that
    else:
        inc = PA = 0.

    # truncate at appropriate rmax
    if rmax is not None:
        idx = np.where(r > rmax)[0][0]
    else:
        idx = np.where(intens < 0)[0][0]
        rmax = r[idx-1]
    
    r = r[:idx]
    intens = intens[:idx]
    
    # to enable interpolation at small radii
    r = np.insert(r, 0, 0)
    intens = np.insert(intens, 0, intens[0]) 
    
    if log: 
        intens[intens<1e-2] = 1e-2
        I = interp1d(r, intens, kind='linear', bounds_error=False, fill_value=1e-2,  assume_sorted=True) 
        # gets rid of unplottably small values in log plot
    else:
        I = interp1d(r, intens, kind='linear', bounds_error=False, fill_value=0,  assume_sorted=True) 

    # Mesh 
    x = np.linspace(-1, 1, nxy) * rmax
    b = np.cos(inc)
    x /= b # inclinination is measured parallel to x-axis, reduces flux appropriately (see p2i)
    y = np.linspace(-1, 1, nxy) * rmax
    X, Y = np.meshgrid(x, y)

    # map intensity onto mesh
    R = np.sqrt(X**2 + Y**2)
    image = I(R)

    if axisym:
        image_s = image

    else:
        # # Add SPIRAL
        image_s = add_spiral(a_or_i, Mth, dustnum, hand, r_planet, SA*np.pi/180, image, x*1/rad2arcsec, y*1/rad2arcsec, band=band)

        # # Add SPIRAL fast (used as a check)
        # # for add_spiral
        # dI_I, r_spiral, dphi = get_spiral(a_or_i, Mth, dustnum, band=band) # r_spiral = 1 corresponds to planet position
        # X, Y = np.meshgrid(x*1/rad2arcsec, y*1/rad2arcsec)
        # R = np.sqrt(X**2 + Y**2)
        # Phi = np.arctan2(Y, X)
        # Nphi, Nr = dI_I.shape
        # dI_I = np.vstack((dI_I, dI_I[-1,:])) # To avoid holes in the 0,2pi boundary, the last row of the data copied in the begining 
        # image_s = add_spiral_fast(dI_I, r_spiral, dphi, hand, r_planet, SA*np.pi/180, image, Phi, R, Nphi, Nr)

        # Rotate the image by PA (anticlockwise)
        image_s = rotate(image_s, PA, reshape=False, order=3)

    if log:
        image_s[image_s < 1e-2] = 1e-2
    else:
        image_s[image_s < 0] = 0

    ### Plotting

    # theme
    sns.set_theme(palette='deep')
    matplotlib.rcParams['mathtext.fontset'] = 'stix'
    matplotlib.rcParams['font.family'] = 'STIXGeneral'
    cmap='inferno'

    fig, ax = plt.subplots()
    # ax.set_title(f'{disk} Spiral-Including Model Disk', pad=20, fontsize=size+1, fontweight='bold')

    if rmax > 0.5:
        step = 0.2
    else:
        step = 0.1

    if log:
        plt.pcolormesh(x*b, y, image_s, cmap=cmap, norm=colors.LogNorm(vmin=1e-2))
        # we use x*b instead of x as we just want to represent spatial scales,
        # so we have to reverse the scaling needed to generate correct images earlier (note x*b = y so really its just a square background grid)
    else: 
        plt.pcolormesh(x*b, y, image_s, cmap=cmap) 

    cbar = plt.colorbar()
    cbar.set_label(r'I [Jy arcsec$^{-2}$]', fontsize=size)
    cbar.ax.tick_params(labelsize=size)
    ax.set_xlabel("RA offset ['']", fontsize=size)
    ax.set_ylabel("Dec offset ['']", fontsize=size)
    ax.set_xlim(rmax,-rmax) # forces RA to run right to left 
    ax.xaxis.set_major_locator(ticker.MultipleLocator(step))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(step))
    ax.set_aspect('equal', adjustable='box')
    ax.tick_params(axis='both', which='major', labelsize=size)
    plt.subplots_adjust(bottom=0.15)  # Adjust the bottom space to a suitable value


    # save file
    if png and log and project:
        filename = '/home/es833/Proj42_results/pretties/' + disk + '_' + str1(r_planet) + 'rp-' + str(round(SA)) + 'deg_' + a_or_i + '-' + Mth + 'Mth-dust' + dustnum + '-' + hand + '_pretty_log_projected.png'
        fig.savefig(filename, bbox_inches='tight', dpi=330)
    
    elif png and project:
        filename = '/home/es833/Proj42_results/pretties/' + disk + '_' + str1(r_planet) + 'rp-' + str(round(SA)) + 'deg_' + a_or_i + '-' + Mth + 'Mth-dust' + dustnum + '-' + hand + '_pretty_projected.png'
        fig.savefig(filename, bbox_inches='tight', dpi=330)
    
    elif png and log:
        filename = '/home/es833/Proj42_results/pretties/' + disk + '_' + str1(r_planet) + 'rp-' + str(round(SA)) + 'deg_' + a_or_i + '-' + Mth + 'Mth-dust' + dustnum + '-' + hand + '_pretty_log.png'
        fig.savefig(filename, bbox_inches='tight', dpi=330)
    
    elif png:
        filename = '/home/es833/Proj42_results/pretties/' + disk + '_' + str1(r_planet) + 'rp-' + str(round(SA)) + 'deg_' + a_or_i + '-' + Mth + 'Mth-dust' + dustnum + '_pretty.png'
        fig.savefig(filename, bbox_inches='tight', dpi=330)
    
    else:
        plt.show()
    plt.close()